# Quick plots - CTSM - Spinup 
## This notebook provides a method to check if spinup simulations have reached steady state.  

In this tutorial you will find steps and instructions to:

1. Load python libraries
2. Locate history files
3. Read in preprocesed data
4. Plot total ecosystem C over time
5. Determine if spinup simulations have reached steady state

------

# 1. Load Datasets

## 1.1 Load Python Libraries

In [ ]:
import os
import time
import datetime

import numpy as np
import nc_time_axis
import pandas as pd
import xarray as xr

from glob import glob
from os.path import join

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from neon_utils import fix_time_h0

<div class="alert alert-block alert-info">
<b>Note:</b> If you are having difficulties importing `nc_time_axis`, run the following line once, then comment it out and restart the kernel by clicking on Kernel --> Restart Kernel in the top left panel in order to ensure the package is installed.  
    
> `pip install nc-time-axis`

</div>


In [ ]:
print('xarray '+xr.__version__) # was working with 2023.5.0

## 1.2 Locate and read in history files 

### 1.2.1 Where are my simulation results?
After your simulations finish, history files are all saved in your `/scratch/NEON_cases/archive/` directory

We can print the cases available to look at using bash magic, `%%bash` or `!` which turns the python cell block below into a bash cell.  

In [ ]:
%%bash
ls ~/scratch/NEON_cases/archive/

<div class="alert alert-block alert-info">
<b>Note</b> you can accomplish the same thing with the following code:

> `!ls ~/scratch/NEON_cases/archive/`
    
</div>

---

### 1.2.2 Point to the directory with history files 
**We'll set the following:**
- site and experiment to look at
- path to our archive directory
- directory with input data (where history files are found)   

By doing this more generally, it makes the script easier to modify for different sites.

<div class="alert alert-block alert-info">
<b>Note</b> You can use data that you generated in other tutorials, or you can look at the example data. If you'd like to use example data, please uncomment the example archive directory line below.
</div>

In [ ]:
neon_site = 'WOOD'  # NEON site we're going to look at
experiment = '.BTRAN.ad' # Can set to ad, postad, or transient

# If you would like to look at your own data, set the path to your archive directory
archive = '~/scratch/NEON_cases/archive'

# If you would like to look at example data, set the path to this archive directory:
# archive = '/scratch/wwieder/NEON_cases/archive'

# This expands the shortcut '~' used above
archive = os.path.realpath(os.path.expanduser(archive)) 

# Create a path to the data folder
data_folder = archive+'/'+neon_site+experiment+'/lnd/hist'
data_folder

**Is this the path for input data, `data_folder`, correct?** 

*HINT:* You can check in the terminal window or using bash magic.

---

### 1.2.3 Create some functions we'll use when opening the data
`preprocess_all` will limit the number of coordinate variables we're reading in. This is an xarray feature that helps save time (and memory resources).

In [ ]:
# Define function to preprocess all variables from the netcdf files
# This just drops an unused coordinate variable (lndgrid) from the dataset
def preprocess_all (ds):
    ds_new= ds.isel(lndgrid=0) 
    return ds_new

Now we have created the functions needed to manipulate our datasets

---

### 1.2.4 List all the files we're going to open
The the monthly history output (**'h0' files**) are written out for NEON cases. 

To open all of these files we're going to need to know their names.  This can be done if we:
- Create an empty list `[]` of simulation files that is
- `.extend`ed with a 
- `sorted` list of files generted with the 
- `glob` function in python of the 
- `*h0*`files in our `data_folder` 

In [ ]:
# Create an empty list of all the file names to extend
sim_files = []
sim_files.extend(sorted(glob(join(data_folder,"*h0.*.nc"))))

print("All simulation files for all years: [", len(sim_files), "files]")
print(sim_files[-1])

---

### 1.2.5 Read in the data

In [ ]:
start = time.time()
print ("Reading in data for "+neon_site)

ds_ctsm = xr.open_mfdataset(sim_files, decode_times=True, combine='by_coords',
                            preprocess=preprocess_all)

end = time.time()
print("Reading all simulation files took:", end-start, "s.")

### 1.2.6 Print the dataset you're working with

In [ ]:
ds_ctsm

---

## 1.3 Plot total ecosystem C over time
We can visually check to see if the simulation achieved steady-state.

In [ ]:
ds_ctsm.TOTECOSYSC.plot()
plt.title(neon_site+experiment) ;

**LOOK**
- Does the model seem to have achieved steady state?
- Why are there periodic oscillations in the total ecosystem C pools?

<div class="alert alert-block alert-success">
<b>Congratulations:</b> 
    
You've quickly looked at total ecosystem carbon stocks to check for steady state. 
    
Are there other variables would you like to look at?
    
</div>
